# Experiment tracking

`SQLiteTracker` provides a powerful and flexible way to track computational (e.g., Machine Learning) experiments using a SQLite database. Allows you to use SQL as the query language, giving you a powerful tool for experiment comparison, and comes with plotting features to compare plots side-by-side and to combine plots for better comparison.

Read more about the motivations in our [blog post](https://ploomber.io/blog/experiment-tracking/), check out the [HN discussion.](https://news.ycombinator.com/item?id=33624018)

This tutorial will walk you through the features with a Machine Learning use case; however, the tracker is generic enough to be used in any other domains.

In [23]:
from sklearn_evaluation import SQLiteTracker

from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [24]:
tracker = SQLiteTracker('my_experiments.db')

In [25]:
X, y = datasets.make_classification(200, 10, n_informative=5, class_sep=0.65)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

models = [RandomForestClassifier(), LogisticRegression(), DecisionTreeClassifier()]

## Training and logging models

In [26]:
for m in models:
    model = type(m).__name__
    print(f'Fitting {model}')


    experiment = tracker.new_experiment()
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # log a dictionary with log_dict
    experiment.log_dict({'accuracy': acc, 'model': model, **m.get_params()})

Fitting RandomForestClassifier
Fitting LogisticRegression
Fitting DecisionTreeClassifier


Or use `.log(key, value)` to log individual values:

In [27]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc = accuracy_score(y_test, y_pred)

experiment = tracker.new_experiment()

# log individual values
experiment.log('accuracy', acc)
experiment.log('model', type(svc).__name__)

_ = experiment.log_dict(svc.get_params())

## Displaying latest experiments

Display the `tracker` object to show last experiments:

In [28]:
tracker

uuid,created,parameters,comment
da87fe76,2022-11-24 18:59:09,"{""accuracy"": 0.7424242424242424, ""model"": ""RandomForestClassifier"", ""bootstrap"": true, ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": ""sqrt"", ""max_leaf_nodes"": null, ""max_samples"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""n_estimators"": 100, ""n_jobs"": null, ""oob_score"": false, ""random_state"": null, ""verbose"": 0, ""warm_start"": false}",
9e693943,2022-11-24 18:59:09,"{""accuracy"": 0.6515151515151515, ""model"": ""LogisticRegression"", ""C"": 1.0, ""class_weight"": null, ""dual"": false, ""fit_intercept"": true, ""intercept_scaling"": 1, ""l1_ratio"": null, ""max_iter"": 100, ""multi_class"": ""auto"", ""n_jobs"": null, ""penalty"": ""l2"", ""random_state"": null, ""solver"": ""lbfgs"", ""tol"": 0.0001, ""verbose"": 0, ""warm_start"": false}",
d44a0bc2,2022-11-24 18:59:09,"{""accuracy"": 0.696969696969697, ""model"": ""DecisionTreeClassifier"", ""ccp_alpha"": 0.0, ""class_weight"": null, ""criterion"": ""gini"", ""max_depth"": null, ""max_features"": null, ""max_leaf_nodes"": null, ""min_impurity_decrease"": 0.0, ""min_samples_leaf"": 1, ""min_samples_split"": 2, ""min_weight_fraction_leaf"": 0.0, ""random_state"": null, ""splitter"": ""best""}",
34f111b8,2022-11-24 18:59:09,"{""accuracy"": 0.7575757575757576, ""model"": ""SVC"", ""C"": 1.0, ""break_ties"": false, ""cache_size"": 200, ""class_weight"": null, ""coef0"": 0.0, ""decision_function_shape"": ""ovr"", ""degree"": 3, ""gamma"": ""scale"", ""kernel"": ""rbf"", ""max_iter"": -1, ""probability"": false, ""random_state"": null, ""shrinking"": true, ""tol"": 0.001, ""verbose"": false}",


## Querying experiments with `.query()`

You can use SQL to query your experiments:

In [29]:
ordered = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
""")
ordered

,model,accuracy
uuid,,
34f111b8,SVC,0.757576
da87fe76,RandomForestClassifier,0.742424
d44a0bc2,DecisionTreeClassifier,0.696970
9e693943,LogisticRegression,0.651515


The query method returns a data frame with "uuid" as the index:

In [30]:
type(ordered)

pandas.core.frame.DataFrame

## Storing plots

You can log a confusion matrix and classification reports:

In [31]:
def fit(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    experiment = tracker.new_experiment()
    experiment.log_dict({'accuracy': acc, 'model': type(model).__name__, **model.get_params()})
    
    # log plots     
    experiment.log_confusion_matrix(y_test, y_pred)
    experiment.log_classification_report(y_test, y_pred)
    
    # log generic figure

fit(model=RandomForestClassifier(n_estimators=100))
fit(model=RandomForestClassifier(n_estimators=10))

In [32]:
tracker.recent(2)

,created,parameters,comment
uuid,,,
5d8c457d,2022-11-24 18:59:12,"{""accuracy"": 0.803030303030303, ""model"": ""Rand...",None
e5705b82,2022-11-24 18:59:11,"{""accuracy"": 0.7575757575757576, ""model"": ""Ran...",None


## Rendering plots in table view

The `.query()` method also allows rendering plots in the table view:

In [33]:
results = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy,
       json_extract(parameters, '$.confusion_matrix') AS cm
FROM experiments
ORDER BY created DESC
LIMIT 2
""", as_frame=False, render_plots=True)

results

uuid,model,accuracy,cm
5d8c457d,RandomForestClassifier,0.803030,
e5705b82,RandomForestClassifier,0.757576,


## Side-by-side comparison

From the `.query()` results, you can extract a given column for a side by side comparison:

In [34]:
results.get("cm")

You can change the labels in the tabs with the `index_by` argument:

In [35]:
results.get("cm", index_by="accuracy")

## Combining plots

With a side-by-side comparison, it might be hard to spot the model performance differents, you can get individual experiments, extract their plots and combine them:

In [43]:
# get the uuids for the latest 2 experiments
uuid1, uuid2 = tracker.recent(2).index

# get the experiments
one = tracker.get(uuid1)
another = tracker.get(uuid2)

In [15]:
one["confusion_matrix"] + another["confusion_matrix"]

In [16]:
one["classification_report"] + another["classification_report"]

Note that plot combination is only supported by plots generated via the `experiment.log_*` and not by the generic `experiment.log_figure` method.

## Adding comments

In [17]:
one.comment('This is some comment')

In [19]:
tracker.query("""
SELECT uuid,
       comment,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
WHERE comment is not NULL
""")

,comment,model,accuracy
uuid,,,
580339db,This is some comment,RandomForestClassifier,0.787879


## Pandas integration

### Getting recent experiments

The recent method also returns a data frame:

In [20]:
df = tracker.recent()
df

,created,parameters,comment
uuid,,,
580339db,2022-11-24 17:48:48,"{""accuracy"": 0.7878787878787878, ""model"": ""Ran...",This is some comment
99888ef3,2022-11-24 17:48:47,"{""accuracy"": 0.7727272727272727, ""model"": ""Ran...",None
2e7b5ad2,2022-11-24 17:48:34,"{""accuracy"": 0.7424242424242424, ""model"": ""SVC...",None
3acd5678,2022-11-24 17:48:32,"{""accuracy"": 0.7878787878787878, ""model"": ""Ran...",None
0f416114,2022-11-24 17:48:32,"{""accuracy"": 0.6363636363636364, ""model"": ""Log...",None


Pass `normalize=True` to convert the nested JSON dictionary into columns:

In [21]:
df = tracker.recent(normalize=True)
df

,created,accuracy,model,bootstrap,ccp_alpha,class_weight,criterion,max_depth,max_features,max_leaf_nodes,...,shrinking,tol,dual,fit_intercept,intercept_scaling,l1_ratio,multi_class,penalty,solver,comment
uuid,,,,,,,,,,,,,,,,,,,,,
580339db,2022-11-24 17:48:48,0.787879,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This is some comment
99888ef3,2022-11-24 17:48:47,0.772727,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2e7b5ad2,2022-11-24 17:48:34,0.742424,SVC,NaN,NaN,None,NaN,NaN,NaN,NaN,...,True,0.0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3acd5678,2022-11-24 17:48:32,0.787879,RandomForestClassifier,True,0.0,None,gini,NaN,sqrt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
0f416114,2022-11-24 17:48:32,0.636364,LogisticRegression,NaN,NaN,None,NaN,NaN,NaN,NaN,...,NaN,0.0001,False,True,1.0,NaN,auto,l2,lbfgs,None


### Querying

You can also use the `.query()` method with `as_frame=True` (default value) to get a `pandas.DataFrame`

In [22]:
df = tracker.query("""
SELECT uuid,
       json_extract(parameters, '$.model') AS model,
       json_extract(parameters, '$.accuracy') AS accuracy
FROM experiments
ORDER BY accuracy DESC
LIMIT 3
""")
df

,model,accuracy
uuid,,
3acd5678,RandomForestClassifier,0.787879
580339db,RandomForestClassifier,0.787879
99888ef3,RandomForestClassifier,0.772727


In [ ]:
# delete our example database
from pathlib import Path
Path('my_experiments.db').unlink()